# InSAR time series analysis with HyP3 and MintPy

This notebook shows how to do time-series analysis using HyP3 product with MintPy. It requires `hyp3_sdk` and `MintPy`:

+ run `conda install --yes -c conda-forge hyp3_sdk ipywidgets` to install `hyp3_sdk`
+ check the [installation page](https://github.com/insarlab/MintPy/blob/main/docs/installation.md) to install `MintPy`

## 0. Initial setup of the notebook

The cell below performs the intial setup of the notebook and must be **run every time the notebook (re)starts**. It imports necessary modules and defines the processing location.

In [ ]:
%matplotlib inline
import os
import numpy as np
import matplotlib.pyplot as plt
# verify mintpy install
try:
    #from mintpy.objects.insar_vs_gps import plot_insar_vs_gps_scatter
    #from mintpy.unwrap_error_phase_closure import plot_num_triplet_with_nonzero_integer_ambiguity
    #from mintpy import workflow, view, tsview, plot_network, plot_transection, plot_coherence_matrix
    from mintpy import view, tsview
except ImportError:
    raise ImportError("Can not import mintpy!")

# utils function
def configure_template_file(outName, CONFIG_TXT): 
    """Write configuration files for MintPy to process HyP3 product"""
    if os.path.isfile(outName):
        with open(outName, "w") as fid:
            fid.write(CONFIG_TXT)
        print('write configuration to file: {}'.format(outName))

    else:
        with open(outName, "a") as fid:
            fid.write("\n" + CONFIG_TXT)
        print('add the following to file: \n{}'.format(outName))

# define the work directory
#work_dir = os.path.abspath(os.path.join(os.getcwd(), 'mintpy'))      #OpenSARLab at ASF
proj_name = 'Ridgecrest'
proj_dir = os.path.join('/media/jzhu4/data/hyp3-mintpy', proj_name)  #Local
hyp3_dir = os.path.join(proj_dir, 'hyp3')
work_dir = os.path.join(proj_dir, 'mintpy')   #Local

if not os.path.isdir(proj_dir):
    os.makedirs(proj_dir)
    print('Create directory: {}'.format(proj_dir))
    
if not os.path.isdir(hyp3_dir):
    os.makedirs(hyp3_dir)
    print('Create directory: {}'.format(hyp3_dir))
    
if not os.path.isdir(work_dir):
    os.makedirs(work_dir)
    print('Create directory: {}'.format(work_dir))
    
os.chdir(work_dir)
print('Go to work directory: {}'.format(work_dir))

## 1. Run MintPy routine workflow `smallbaselineApp.py`

### 1.1 Prepare the template file

In [ ]:
CONFIG_TXT = f'''# vim: set filetype=cfg:
mintpy.load.processor        = hyp3
##---------interferogram datasets:
mintpy.load.unwFile          = {hyp3_dir}/*/*unw_phase_clip.tif
mintpy.load.corFile          = {hyp3_dir}/*/*corr_clip.tif
##---------geometry datasets:
mintpy.load.demFile          = {hyp3_dir}/*/*dem_clip.tif
mintpy.load.incAngleFile     = {hyp3_dir}/*/*inc_map_clip.tif
'''
print(CONFIG_TXT)
configName = os.path.join(work_dir, "{}.txt".format(proj_name))
configure_template_file(configName, CONFIG_TXT)

### 1.2 Load the data produced from HyP3

The example dataset is from 2019 Ridgecrest, CA earthquake. The dataset can be obtained through either downloading from the stagged server or producing with hyp3-sdk. As far as producing data from hyp3-sdk, we provide the prep_ts_hyp3 notebook at the tutorial directory of (https://github.com/ASFHyP3/hyp3-docs/tree/develop/docs ).

In [ ]:
# verify / prepare input dataset

os.chdir(hyp3_dir)

use_staged_data = True

zip_file_name ='Ridgecrest.zip'

if all(os.path.isfile(os.path.join(work_dir, 'inputs', i)) for i in ['ifgramStack.h5', 'geometryGeo.h5']):
    print("Required inputs for mintpy already exists.")

else:
    if use_staged_data:
        # Check if a stage file from S3 already exist, if not try and download it
        zip_file = os.path.join(hyp3_dir, zip_file_name)
        if not os.path.isfile(zip_file):
            !wget https://jzhu-hyp3-dev.s3.us-west-2.amazonaws.com/hyp3-mintpy/{zip_file_name}
            #!aws s3 cp s3://jzhu-hyp3-dev/hyp3-mintpy-example/{zip_file_name} {zip_file_name}
        # verify if download was succesfull
        if os.path.isfile(zip_file_name):
            import zipfile, glob
            
            with zipfile.ZipFile(zip_file, 'r') as fzip:
                fzip.extractall(hyp3_dir)
            # unzip zip files extracted from the zip_file
            files = glob.glob("./????_*.zip")
            for file in files:
                with zipfile.ZipFile(file) as f:
                    f.extractall(hyp3_dir)
                    
            print('S3 pre-staged data retrieval was successfull')

        else:
            msg = 'No staged data. Setting use_staged_data = False and re-run this cell.'
            print(msg)

    else:
        print("Using HyP3-sdk to download and prepare the input data for MintPy")
        print("please refer the notebook")
        os.chdir(os.path.dirname(work_dir))

### 1.3 Run Time-series Analysis application

In [ ]:
! smallbaselineApp.py --work-dir {work_dir}  {configName}

### 1.4 Display the analysis results

There are a few scripts used to display the analysis results. There are in the MINTPY_HOME/mintpy. Here we show two majoy disaply scripts.

In [ ]:
os.chdir(proj_dir)

In [ ]:
view.main(['mintpy/velocity.h5'])

In [ ]:
tsview.main(['mintpy/timeseries.h5'])